Objective for this lab is to:

1. Use Tools (Push notification tool, custom made tools to record user contact details and quesstions that LLM cant answer and send them as push notifications)
2. Learn to deploy gradio app on huggingface spaces 

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import requests
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
# Function to send a push notification using Pushover
def push(message):
    print(f"Push: {message}")
    payload = {"user":pushover_user, "token":pushover_token, "message":message}
    requests.post(pushover_url, data=payload)

In [6]:
push("HEY!")

Push: HEY!


In [7]:
# These two would be manually coded tools that we would provide to the agent.

def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes: {notes}")
    return {"recorded":"ok"}


def record_unknown_question(question):
    push(f"Recording {question} asked that I could not answer")
    return {"recorded":"ok"}

In [12]:
record_user_details_json = {
    "name": "record_user_details",
    "description":"Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of the user"
            }, 
            "name": {
                "type": "string",
                "description": "The name of the user, if provided"
            },
            "notes": {
                "type": "string",
                "description": "Any additional notes provided by the user"
            }
        },      
        "required": ["email"],
        "additionalProperties": False
    
}}




record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [13]:
tools = [{"type": "function", "function": record_user_details_json},
         {"type": "function", "function": record_unknown_question_json}]    

In [19]:
# Function below would be able to run the tools provided as lists (Its the IF statement version)

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}, flush = True")


        # This is the AI i.e. IF statement that would run the tools

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        result.append({"role":"tool","content":json.dumps(result), "tool_call_id":tool_call.id})
    return results

In [23]:
# Thsi is how you can handle those if statements in a more generic way using globals
globals()["record_unknown_question"]("This a test question")

Push: Recording This a test question asked that I could not answer


{'recorded': 'ok'}